In [1]:
# Dependencies 
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from selenium import webdriver
import pandas as pd

In [2]:
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
# Get url to scape
url= "https://mars.nasa.gov/news/"
browser.visit(url)
html = browser.html

# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')

In [5]:
# Collect the latest news title and paragraph
title = soup.find_all('div', class_='content_title')[0].text
p = soup.find_all('div', class_='article_teaser_body')[0].text

# Print the titles and paragraphs 
print(title)
print("----------------------")
print(p)

Mars Now
----------------------
Did you know NASA's next Mars rover has a nearly identical sibling on Earth for testing? Even better, it's about to roll for the first time through a replica Martian landscape.
